In [11]:
import requests


BASE_URL = "http://localhost:8000"

# Query parameters for the request
params = {
    "name": "Testing",
    "email": "lopesdasilva99999@gmail.com"
}

# Make the request
try:
    response = requests.get(f"{BASE_URL}/portfolio/search", params=params)
    response.raise_for_status()  # Raise an exception for HTTP errors
    data = response.json()
    #print("Response:", data)
except requests.HTTPStatusError as e:
    print(f"HTTP error occurred: {e.response.status_code} - {e.response.text}")
except Exception as e:
    print(f"An error occurred: {e}")





port_info = data['stocks']
print(port_info)




[{'symbol': 'NKE', 'quantity': 4, 'buyPrice': 6.0, 'buyDate': '2024-01-01'}, {'symbol': 'AAPL', 'quantity': 2, 'buyPrice': 3.0, 'buyDate': '2024-01-01'}, {'symbol': 'NVDA', 'quantity': 7, 'buyPrice': 8.0, 'buyDate': '2024-01-01'}]


In [17]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# Portfolio data
portfolio = [
    {'symbol': 'NKE', 'quantity': 4, 'buyPrice': 6.0, 'buyDate': '2024-01-01'},
    {'symbol': 'AAPL', 'quantity': 2, 'buyPrice': 3.0, 'buyDate': '2024-01-01'},
    {'symbol': 'NVDA', 'quantity': 7, 'buyPrice': 8.0, 'buyDate': '2024-01-01'}
]

# Function to fetch sector classifications
def fetch_sector_classification(portfolio):
    sector_classification = {}
    for stock in portfolio:
        symbol = stock['symbol']
        try:
            ticker_info = yf.Ticker(symbol).info  # Retrieve stock info
            sector = ticker_info.get('sector', 'Unknown')  # Fetch sector
            sector_classification[symbol] = sector
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            sector_classification[symbol] = 'Unknown'
    return sector_classification

# Fetch current stock data
def fetch_current_prices(portfolio):
    prices = {}
    for stock in portfolio:
        symbol = stock['symbol']
        try:
            data = yf.Ticker(symbol).history(period="1d")  # Get latest price
            current_price = data['Close'].iloc[-1]  # Close price
            prices[symbol] = current_price
        except Exception as e:
            print(f"Error fetching price for {symbol}: {e}")
            prices[symbol] = 0
    return prices

# Calculate portfolio value and sector allocation
def calculate_portfolio_stats(portfolio, prices, sectors):
    total_value = 0
    sector_values = {}

    for stock in portfolio:
        symbol = stock['symbol']
        quantity = stock['quantity']
        current_price = prices[symbol]
        stock_value = quantity * current_price

        # Update total and sector values
        total_value += stock_value
        sector = sectors.get(symbol, 'Unknown')
        if sector not in sector_values:
            sector_values[sector] = 0
        sector_values[sector] += stock_value

    # Calculate sector allocations
    sector_allocations = {sector: (value / total_value) * 100 for sector, value in sector_values.items()}
    return total_value, sector_allocations

# Calculate portfolio returns
def calculate_returns(portfolio, prices):
    total_cost = sum(stock['quantity'] * stock['buyPrice'] for stock in portfolio)
    current_value = sum(stock['quantity'] * prices[stock['symbol']] for stock in portfolio)
    return ((current_value - total_cost) / total_cost) * 100

# Plot sector allocation
def plot_sector_allocation(sector_allocations):
    labels = sector_allocations.keys()
    sizes = sector_allocations.values()
    plt.figure(figsize=(8, 6))
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
    plt.title('Sector Allocation')
    plt.show()


# Fetch sector classifications
sectors = fetch_sector_classification(portfolio)
print("Sector Classification:", sectors)

# Fetch current prices
current_prices = fetch_current_prices(portfolio)
print("Current Prices:", current_prices)

# Calculate portfolio stats
total_value, sector_allocations = calculate_portfolio_stats(portfolio, current_prices, sectors)

# Calculate portfolio returns
portfolio_return = calculate_returns(portfolio, current_prices)

# Display results
print(f"Portfolio Total Value: ${total_value:.2f}")
print(f"Sector Allocations: {sector_allocations}")
print(f"Portfolio Return: {portfolio_return:.2f}%")

# Plot sector allocation
plot_sector_allocation(sector_allocations)


Sector Classification: {'NKE': 'Consumer Cyclical', 'AAPL': 'Technology', 'NVDA': 'Technology'}
Current Prices: {'NKE': 78.0, 'AAPL': 253.47999572753906, 'NVDA': 130.38999938964844}
Portfolio Total Value: $1731.69
Sector Allocations: [('Consumer Cyclical', 18.017081712622833), ('Technology', 81.98291828737717)]
Portfolio Return: 1913.59%


AttributeError: 'list' object has no attribute 'keys'